In [1]:
import tensorflow as tf
from tensorflow.keras.losses import SparseCategoricalCrossentropy

from Transformer import Transformer
from Utils import Dataset

2024-06-14 22:29:02.453964: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-14 22:29:02.454050: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-14 22:29:02.454092: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-14 22:29:02.462191: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
MAX_LENGTH = 100
BUFFER_SIZE = 20000
BATCH_SIZE = 64

In [3]:
dataset = Dataset("ner.csv", MAX_LENGTH, BATCH_SIZE, BUFFER_SIZE)


2024-06-14 22:29:04.666912: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-14 22:29:04.700272: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-14 22:29:04.700321: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-14 22:29:04.703922: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-06-14 22:29:04.703978: I tensorflow/compile

In [4]:
train_batches, test_batches = dataset.get_data_generators()

In [5]:
num_layers = 2
d_model = 64
dense_hidden_num_units = 256
num_heads = 8

transformer = Transformer(num_layers, d_model, num_heads, dense_hidden_num_units, dataset.get_vocab_size(), 20, MAX_LENGTH)

transformer.compile(
    optimizer="rmsprop",
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)



In [6]:
history = transformer.fit(
    train_batches,
    epochs=5,
    validation_data=test_batches
)

Epoch 1/5


2024-06-14 22:29:17.280942: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8907
2024-06-14 22:29:17.690129: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f2d48e2a020 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-06-14 22:29:17.690186: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2024-06-14 22:29:17.694131: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-14 22:29:17.749576: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


600/600 [==============================] - 70s 104ms/step - loss: 0.1728 - accuracy: 0.9537 - val_loss: 0.1339 - val_accuracy: 0.9632
Epoch 2/5
600/600 [==============================] - 57s 95ms/step - loss: 0.1305 - accuracy: 0.9645 - val_loss: 0.1219 - val_accuracy: 0.9673
Epoch 3/5
600/600 [==============================] - 57s 95ms/step - loss: 0.1188 - accuracy: 0.9667 - val_loss: 0.1130 - val_accuracy: 0.9659
Epoch 4/5
600/600 [==============================] - 57s 95ms/step - loss: 0.0696 - accuracy: 0.9774 - val_loss: 0.0594 - val_accuracy: 0.9814
Epoch 5/5
600/600 [==============================] - 56s 93ms/step - loss: 0.0366 - accuracy: 0.9886 - val_loss: 0.0319 - val_accuracy: 0.9901


In [7]:
class NamedEntityRecognizer(tf.Module):
  def __init__(self, transformer):
    self.transformer = transformer

  def __call__(self, sentence, max_length=MAX_LENGTH):
    sentence = dataset.vectorize(sentence)

    encoder_input = sentence

    start_end = dataset.encode_label(["[START]","[END]"])
    start = start_end[0][tf.newaxis]
    end = start_end[1][tf.newaxis]

    output_array = tf.TensorArray(dtype=tf.int64, size=0, dynamic_size=True)
    output_array = output_array.write(0, start)

    for i in tf.range(max_length):
      output = tf.transpose(output_array.stack())
      predictions = self.transformer([encoder_input, output], training=False)

      predictions = predictions[:, -1:, :]  
      predicted_id = tf.argmax(predictions, axis=-1)

      output_array = output_array.write(i+1, predicted_id[0])

      if predicted_id == end:
        break

    output = tf.transpose(output_array.stack())
    text = [dataset.inverse_transform_label(out) for out in output]

    return text


In [8]:
translator = NamedEntityRecognizer(transformer)

In [11]:
for (s,t), t_l in train_batches.take(1):
    break

sentence = [dataset.decode(s[4])]

translated_text = translator(
    sentence
)
translated_text

[array(['[START]', 'B-gpe', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O',
        'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',
        'O', 'O', 'O', 'O', 'O', 'O', '[END]'], dtype='<U7')]

In [12]:
sentence

["pakistani officials say u.s. drone aircraft have fired missiles into a suspected militant compound in the country 's northwest tribal belt , killing at least 10 militants .                                                                        "]

In [13]:
dataset.inverse_transform_label(t[4])

array(['[START]', 'B-gpe', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O',
       'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',
       'O', 'O', 'O', 'O', 'O', 'O', '[END]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',
       '[PAD]', '[PAD]', '[PAD]', '[PAD]'], dtype='<U7')